# Rigorous Thermodynamic Decomposition of Salt Effects on the Polymerization of Polyethylene Glycol
Stefan Hervø-Hansen<sup>a,*</sup>, Kazuo Yamada<sup>a,*</sup>, Jan Heyda<sup>b,*</sup>, and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>b</sup> Department of Physical Chemistry, University of Chemistry and Technology, Prague CZ-16628, Czech Republic.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp, k.yamada@cheng.es.osaka-u.ac.jp, heydaj@vscht.cz, and nobuyuki@cheng.es.osaka-u.ac.jp.

## Part 1: Simulations


### Introduction


### Methods & Materials
Molecular dynamics simulations are conducted using the openMM (7.7.0)[<sup>2</sup>](#fn2) software package modded with the openmmtools[<sup>3</sup>](#fn3) and parmed[<sup>4</sup>](#fn4) packages. For the simulation of PEG a CHARMM derived force field (C35r) was utilized, which has previously been able to reproduce hydrodynamic radii and shape anisotropy of PEG.[<sup>5</sup>](#fn5) The PEG force field was employed in combination with the SPC/E force field for water[<sup>6</sup>](#fn6) and optimized ion parameters for sodium thiocyanate and sodium chloride.
The isothermal-isobaric ensemble will be sampled using a combination of a geodesic Langevin integrator[<sup>8</sup>](#fn8) and a Monte Carlo barostat[<sup>9,</sup>](#fn9)[<sup>10</sup>](#fn10). The trajectories was analyzed using MDtraj[<sup>11</sup>](#fn11) for structural properties, while ERmod[<sup>12</sup>](#fn12) be utilized for the calculation of solvation free energies and can be found in the [Part 2 Jupyter notebook](Analysis.ipynb).

### References
2. <span id="fn2"> Eastman P, et al. (2017) OpenMM 7: Rapid development of high performance algorithms for molecular dynamics. PLOS Computational Biology 13(7):e1005659.</span><br>
3. <span id="fn3"> https://github.com/choderalab/openmmtools</span><br>
4. <span id="fn4"> https://github.com/ParmEd/ParmEd </span><br>
5. <span id="fn5"> Lee H, Venable RM, Mackerell Jr AD, Pastor RW (2008) Molecular dynamics studies of polyethylene oxide and polyethylene glycol: hydrodynamic radius and shape anisotropy. Biophys J. 95(4):1590-1599. </span><br>
6. <span id="fn6"> Berendsen HJC, Grigera JR, Straatsma TP (1987) The missing term in effective pair potentials. The Journal of Physical Chemistry 91(24):6269–6271. </span><br>
8. <span id="fn8"> Leimkuhler B, Matthews C (2016) Efficient molecular dynamics using geodesic integration and solvent–solute splitting. Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences 472(2189):20160138. </span><br>
9. <span id="fn9"> Chow K-H, Ferguson DM (1995) Isothermal-isobaric molecular dynamics simulations with Monte Carlo volume sampling. Computer Physics Communications 91(1–3):283–289. </span><br>
10. <span id="fn10"> Åqvist J, Wennerström P, Nervall M, Bjelic S, Brandsdal BO (2004) Molecular dynamics simulations of water and biomolecules with a Monte Carlo constant pressure algorithm. Chemical Physics Letters 384(4–6):288–294. </span><br>
11. <span id="fn11"> McGibbon RT, et al. (2015) MDTraj: A Modern Open Library for the Analysis of Molecular Dynamics Trajectories. Biophysical Journal 109(8):1528–1532. </span><br>
12. <span id="fn12"> Sakuraba S, Matubayasi N (2014) Ermod: Fast and versatile computation software for solvation free energy with approximate theory of solutions. Journal of Computational Chemistry 35(21):1592–1608. </span><br>


## Import of Python Modules

In [ ]:
# Notebook dependent libs
import parmed as pmd
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import os, re, time
from shutil import copyfile
import pandas as pd
from scipy import stats
from distutils.spawn import find_executable
from IPython.display import IFrame

# Simulation specific libs
import sys
from openmm import app
import openmm as mm
import openmmtools as mmtools
import parmed as pmd
import Auxiliary.Modded_Simulations as mmmod

# Check for external programs
if None in [find_executable('packmol'), find_executable('perl')]:
    print('WARNING: External program missing!')
    

homedir = !pwd
homedir = homedir[0]
print(homedir)

## Molecular Dynamics Simulations

### Simulation settings
For the calculation of solvation free energies, we need to simulate the solvated state ($\lambda=1$) and the reference state ($\lambda=0$), with the duration of the simulation determined by together with the output frequency for the saving configurations and other thermodynamic properties for statistical evaluation, all of which is determined by the `states` variable. The molecular dynamics simulations have been set up such that one can run `Nruns` independent molecular dynamics simulations. The system composition of salts and their concentrations are controlled by the variables `salts` and `concentrations` respectively.

In order to include the effect of flexibility of the solute, we additionally need to generate an ensemble of the solute in a vacuum. In order to do so, the variable `GENERATE_SOLUTE` has been introduced and must be set to True. The vacuum simulations are generated from the `No_salt` entry input files in the `salts` variable and are therefore <i>required</i> before the vacuum inputs can be generated. The structural ensemble is constructed in the canonical ($NVT$) ensemble using molecular dynamics in which the forces and energies are evaluated without cutoffs.

In [ ]:
# State of simulations, (outFreq is steps per frame)
states = {                                                               
          'sol': {'Nsteps': 50000000, 'OutFreq': 2500, 'DO_SIMULATED_TEMPERING': True}, # 100 nanoseconds, 20000 frames
          'ref': {'Nsteps': 500000,   'OutFreq': 500, 'DO_SIMULATED_TEMPERING': False}, # 1 nanoseconds,  1000 frames
         }

# Simulated Tempering Settings
ST_settings = {'numTemperatures': 15,        # Number of temperatures with exponential spacing
               'minTemperature': 298.15,     # Minimum temperature. cf. Langevin integrator temperature
               'maxTemperature': 450,        # Maximum temperature. Aim to unfold PEO
               'reportInterval': 2500,       # Reporting interval of weights and temperature.
               'reportFile': 'Tempering.dat' # Name of the file where the data are stores.
              }

nmers = [2, 4, 6, 8, 15, 36]

salts = {
         'No_salt': {'Cation': 'No_', 'Anion': 'salt' },
#         'NaCl'   : {'Cation': 'Na' , 'Anin': 'Cl'   },
#         'CsCl'   : {'Cation': 'Cs' , 'Anion': 'Cl'   },
#         'NaSCN'  : {'Cation': 'Na' , 'Anion': 'SCN'  },
#         'CsSCN'  : {'Cation': 'Cs' , 'Anion': 'SCN'  },
        }

# Approximate concentrations of salts with
concentrations = {
                  0.00: {'PEG': 1, 'Water': 3000, 'Cation': 0,   'Anion': 0},
                  1.00: {'PEG': 1, 'Water': 3000, 'Cation': 54,  'Anion': 54},
#                  2.00: {'PEG': 1, 'Water': 3000, 'Cation': 109, 'Anion': 109},
                 }

#* Calculated by hand *#
# Cube length ≈ 45 Ångstroms. Round up to 50 and then adjust to correct volume with barostat.

### Construction of topology (.top) and structure (.pdb) files
Fully automated construction of topologies files in gromacs format and initial configurations using packmol. No major important parameters to edit in the following code.

In [ ]:
packmol_script="""
tolerance 2.0
filetype pdb
output PEG_{nmer}_{cation}{anion}_sol.pdb

add_box_sides 0.5

structure {homedir}/PDB_files/PEO-{nmer}-mer.pdb
        number {N_PEG}
        fixed 25. 25. 25. 0. 0. 0.
        centerofmass
end structure

{salt}structure {homedir}/PDB_files/{anion}.pdb
{salt}        number {N_anion}
{salt}        inside cube 0. 0. 0. 50.
{salt}end structure

{salt}structure {homedir}/PDB_files/{cation}.pdb
{salt}        number {N_cation}
{salt}        inside cube 0. 0. 0. 50.
{salt}end structure

structure {homedir}/PDB_files/water.pdb
        number {N_wat}
        inside cube 0. 0. 0. 50.
end structure
"""

topology="""
[ system ]
; Name
Peg in {cation}{anion} {conc} M aqueous solution.

; Include main forcefield parameters
#include "{homedir}/Force_fields/forcefield.itp"

;include custom forcefield parameters
#include "{homedir}/Force_fields/peg-forcefield.itp"
#include "{homedir}/Force_fields/peg{nmer}mer.itp"
#include "{homedir}/Force_fields/spce.itp"
#include "{homedir}/Force_fields/SCN.itp"
#include "{homedir}/Force_fields/ions.itp"

[ molecules ]
; Compound         #mols
PEG{nmer}               {N_PEG}
{salt}{anion}      {N_anion}
{salt}{cation}     {N_cation}
SOL                {N_wat}
"""

In [ ]:
#######################################
####### GENERATE SOLUTE OPTION ########
# Generate Solute vacuum simulations  #
# from simulations files with no salt #
GENERATE_SOLUTE = False                
#######################################

%cd -q $homedir
for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir, salt in salts.items():
        for conc, Nparticles in concentrations.items():
            concdir = '{0:.2f}'.format(conc)
            if conc == 0 and saltdir == 'No_salt':
                %cd -q $homedir/Simulations/$nmerdir/$saltdir
            elif conc != 0 and saltdir == 'No_salt':
                continue
            else:
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$concdir
                
            # Packmol Input
            with open('packmol.in', 'w') as text_file:
                # Fix for no salt
                if conc == 0:
                    saltFix='#'
                else:
                    saltFix=''
                text_file.write(packmol_script.format(N_PEG=Nparticles['PEG'], nmer=nmer, salt=saltFix,
                                                      N_cation=Nparticles['Cation'], N_anion=Nparticles['Anion'],
                                                      cation=salt['Cation'], anion=salt['Anion'],
                                                      N_wat=Nparticles['Water'], homedir=homedir))
            !packmol < packmol.in
            
            # Topology input
            with open('PEG_{nmer}_{salt}_sol.top'.format(nmer=nmer, salt=saltdir), 'w') as text_file:
                # Fix for no salt
                if conc == 0:
                    saltFix=';'
                else:
                    saltFix=''
                text_file.write(topology.format(N_PEG=Nparticles['PEG'], nmer=nmer, salt=saltFix,
                                                N_cation=Nparticles['Cation'], N_anion=Nparticles['Anion'],
                                                cation=salt['Cation'], anion=salt['Anion'], conc=concdir,
                                                N_wat=Nparticles['Water'], homedir=homedir))
            
            # Collect it all for λ=1:
            mol = pmd.load_file('PEG_{nmer}_{salt}_sol.top'.format(nmer=nmer, salt=saltdir),
                                xyz='PEG_{nmer}_{salt}_sol.pdb'.format(nmer=nmer, salt=saltdir))
            mol.save('PEG_{nmer}_{salt}_sol.top'.format(nmer=nmer, salt=saltdir), overwrite=True)
            
            # Generate files for λ=0:
            mol.strip(':PGH,:PGM,:PGT')
            mol.save('PEG_{nmer}_{salt}_ref.top'.format(nmer=nmer, salt=saltdir), overwrite=True)
            mol.save('PEG_{nmer}_{salt}_ref.pdb'.format(nmer=nmer, salt=saltdir), overwrite=True)
            print('Wrote initial configurations and topology files to'+os.getcwd())
            
    # Generate files for vacuum simulation for each nmer:
    if GENERATE_SOLUTE:
        %cd -q $homedir/Simulations/$nmerdir
        mol = pmd.load_file('No_salt/PEG_{nmer}_No_salt_sol.top'.format(nmer=nmer),
                            xyz='No_salt/PEG_{nmer}_No_salt_sol.pdb'.format(nmer=nmer))
        mol.strip(':SOL')
        mol.save('Solute/PEG_{nmer}_vacuum.top'.format(nmer=nmer), overwrite=True)
        mol.save('Solute/PEG_{nmer}_vacuum.pdb'.format(nmer=nmer), overwrite=True)
        print('Wrote initial configurations and topology files to'+os.getcwd()+'/Solute')

### Simulation setup using OpenMM
Fully automated construction of simulation input files using python API for OpenMM. In the following one can edit the integration scheme and its parameters set in the variable `integrator` as well as editing the barostat as currently determined from `mm.MonteCarloBarostat`. Additionally one may change the non-bonded methods and their cutoffs in the `system` variable with the option of adding Lennard-Jones switching functions via the `forces` variable. Finally one may edit the number of minimization (`sim.minimizeEnergy`) and equilibration steps conducted as well as choosing whether the simulation should be conducted on GPUs or CPUs via the `platform` and `properties` variables.

In [ ]:
# openMM script for solution and reference simulations
openmm_script="""# Imports
import sys
import os
from openmm import app
import openmm as mm
import openmmtools as mmtools
from parmed import load_file, unit as u
from mdtraj.reporters import XTCReporter

DO_SIMULATED_TEMPERING = {DO_ST}

print('Loading initial configuration and toplogy')
init_conf = load_file('PEG_{nmer}_{salt}_{state}.top', xyz='PEG_{nmer}_{salt}_{state}.pdb')

# Creating system
print('Creating OpenMM System')
system = init_conf.createSystem(nonbondedMethod=app.PME, ewaldErrorTolerance=0.00001,
                                nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds)

# Calculating total mass of system
total_mass = 0
for i in range(system.getNumParticles()):
    total_mass += system.getParticleMass(i).value_in_unit(u.dalton)
total_mass *= u.dalton

# Temperature-coupling by geodesic Langevin integrator (NVT)
integrator = mmtools.integrators.GeodesicBAOABIntegrator(K_r = 3,
                                                         temperature = 298.15*u.kelvin,
                                                         collision_rate = 1.0/u.picoseconds,
                                                         timestep = 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

# Add LJ switching functions
forces = {{system.getForce(index).__class__.__name__: 
          system.getForce(index) for index in range(system.getNumForces())}}
forces['NonbondedForce'].setUseSwitchingFunction(True)
forces['NonbondedForce'].setSwitchingDistance(1*u.nanometer)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0,1'}}

# Create the Simulation object
sim = app.Simulation(init_conf.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(init_conf.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=1000000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(250000)  # 250000*2 fs = 0.5 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output_{state}.dat', {outFreq}, totalSteps={Nsteps}+250000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory_{state}.xtc', reportInterval={outFreq}, append=False))

if DO_SIMULATED_TEMPERING:
    sim = app.SimulatedTempering(sim, numTemperatures={numT}, minTemperature={minT}, maxTemperature={maxT},
                                 tempChangeInterval=25, reportInterval={outT}, reportFile='{fileT}')

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
try:
    print('''
PARTICLE MESH EWALD PARAMETERS (Conventional MD)
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
except:
    print('''
PARTICLE MESH EWALD PARAMETERS (Simulated tempering)
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.simulation.context)))
"""

# openMM script for vacuum simulations
openmm_vacuum_script="""# Imports
import sys
import os
from openmm import app
import openmm as mm
import openmmtools as mmtools
from parmed import load_file, unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
init_conf = load_file('PEG_{nmer}_vacuum.top', xyz='PEG_{nmer}_vacuum.pdb')

# Creating system
print('Creating OpenMM System')
system = init_conf.createSystem(nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

# Calculating total mass of system
total_mass = 0
for i in range(system.getNumParticles()):
    total_mass += system.getParticleMass(i).value_in_unit(u.dalton)
total_mass *= u.dalton

# Temperature-coupling by geodesic Langevin integrator (NVT)
integrator = mmtools.integrators.GeodesicBAOABIntegrator(K_r = 3,
                                                         temperature = 298.15*u.kelvin,
                                                         collision_rate = 1.0/u.picoseconds,
                                                         timestep = 2.0*u.femtoseconds)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0,1'}}

# Create the Simulation object
sim = app.Simulation(init_conf.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(init_conf.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=1000000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(100000)  # 100000*1 fs = 0.1 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+100000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS (Conventional MD)
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
"""

In [ ]:
%cd -q $homedir
N_simulations = 0
for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir, salt in salts.items():
        for conc in concentrations:
            concdir = '{0:.2f}'.format(conc)
            if conc == 0 and saltdir == 'No_salt':
                %cd -q $homedir/Simulations/$nmerdir/$saltdir
            elif conc != 0 and saltdir == 'No_salt':
                continue
            else:
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$concdir
            for state, settings in states.items():
                
                with open('openMM_{state}.py'.format(state=state), 'w') as text_file:
                    text_file.write(openmm_script.format(state=state, Nsteps=settings['Nsteps'], nmer=nmer,
                                                         outFreq=settings['OutFreq'], salt=saltdir,
                                                         minT=ST_settings['minTemperature'], maxT=ST_settings['maxTemperature'],
                                                         numT=ST_settings['numTemperatures'], outT=ST_settings['reportInterval'],
                                                         fileT=ST_settings['reportFile'], DO_ST=settings['DO_SIMULATED_TEMPERING']))
                N_simulations+=1
            print('Wrote run_openMM.py files to '+os.getcwd())
    if GENERATE_SOLUTE:
        %cd -q $homedir/Simulations/$nmerdir/Solute
        with open('openMM.py', 'w') as text_file:
            text_file.write(openmm_vacuum_script.format(Nsteps=10000000, nmer=nmer, outFreq=1000))
            N_simulations+=1
            print('Wrote run_openMM.py files to '+os.getcwd())

print('Simulations about to be submitted: {}'.format(N_simulations))

### Submit script
Submit script for servers employing job scheduling. The below example is utilizing PBS (for a quick guide see [here](https://latisresearch.umn.edu/creating-a-PBS-script)). However the code below may be edited to utilize Slurm instead (documentation [here](https://slurm.schedmd.com)) by changing the variable `submit_script` and by executing the commands `!sbatch submit_sol.pbs` and `!sbatch submit_ref.pbs` instead of `qsub`.

In [ ]:
submit_script="""#!/bin/bash
#PBS -l nodes=1:ppn=36:nu-g               # 1 node, 36 cores, GPU node, 2 gpu.
#PBS -N PEG-{nmer}_{conc}_M_{salt}_{state}  # Name of job
#PBS -e run_{state}.err               # error output
#PBS -o run_{state}.out               # output file name

source ~/.bashrc
source ~/.bash_profile
cd {path}

python openMM_{state}.py"""

submit_vacuum_script="""#!/bin/bash
#PBS -l nodes=1:ppn=36:nu-g         # 1 node, 36 cores, GPU node, 2 gpu.
#PBS -N PEG-{nmer}_Solute             # Name of job
#PBS -e run.err               # error output
#PBS -o run.out               # output file name

source ~/.bashrc
source ~/.bash_profile
cd {path}

python openMM.py"""

In [ ]:
for nmer in nmers:
    nmerdir = 'PEG{}mer'.format(nmer)
    for saltdir in salts:
        for conc in concentrations:
            concdir = '{0:.2f}'.format(conc)
            if conc == 0 and saltdir == 'No_salt':
                %cd -q $homedir/Simulations/$nmerdir/$saltdir
            elif conc != 0 and saltdir == 'No_salt':
                continue
            else:
                %cd -q $homedir/Simulations/$nmerdir/$saltdir/$concdir
            for state in states:
                with open('submit_{state}.pbs'.format(state=state), 'w') as text_file:
                    text_file.write(submit_script.format(conc=conc, state=state, nmer=nmer,
                                                     path=os.getcwd(), salt=saltdir))
            !qsub submit_sol.pbs
            time.sleep(1) # Safety in submission of jobs: can cause problems if too fast
            !qsub submit_ref.pbs
            time.sleep(1) # Safety in submission of jobs: can cause problems if too fast
            
    if GENERATE_SOLUTE:
        %cd -q $homedir/Simulations/$nmerdir/Solute
        with open('submit.pbs', 'w') as text_file:
            text_file.write(submit_vacuum_script.format(nmer=nmer, path=os.getcwd()))
        !qsub submit.pbs
        time.sleep(1) # Safety in submission of jobs: can cause problems if too fast

## Trajectory reweighting

# RESEARCH NOTES AND TESTING AREA (WILL BE DELETED UPON FINISHING)

http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.System.html#simtk.openmm.openmm.System.addConstraint (fix SCN molecules)

http://docs.openmm.org/7.4.0/api-python/generated/simtk.openmm.app.simulatedtempering.SimulatedTempering.html (to try generate a more overlapping space)

### Checking SCN ff

### Unit test for SimulatedTempering

In [1]:
%%time
import unittest
from openmm import *
from openmm.app import *
from openmm.unit import *


prmtop = AmberPrmtopFile('/home/stefan/openmm/wrappers/python/tests/systems/alanine-dipeptide-explicit.prmtop')
inpcrd = AmberInpcrdFile('/home/stefan/openmm/wrappers/python/tests/systems/alanine-dipeptide-explicit.inpcrd')
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
mcBarostat = MonteCarloBarostat(1*bar, 100*kelvin, 2)
system.addForce(mcBarostat)
integrator = LangevinIntegrator(100*kelvin, 1/picosecond, 0.001*picosecond)
simulation = Simulation(topology, system, integrator, Platform.getPlatformByName('Reference'))
simulation.context.setPositions(inpcrd.positions)
simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

assert 100*kelvin == integrator.getTemperature()
assert 100*kelvin == simulation.context.getParameter('MonteCarloTemperature')*kelvin
st = SimulatedTempering(simulation, numTemperatures=10, minTemperature=200*kelvin, maxTemperature=400*kelvin, tempChangeInterval=4, reportInterval=10000)
assert 200*kelvin == integrator.getTemperature()
assert 200*kelvin == simulation.context.getParameter('MonteCarloTemperature')*kelvin
assert 10 == len(st.temperatures)
assert 200*kelvin == st.temperatures[0]
assert 400*kelvin == st.temperatures[-1]

# We let the simulation run and assert at every step T(mcBarostat) == T(integrator) == T(tempering)
for i in range(50):
    st.step(2)
    print(st.temperatures[st.currentTemperature])
    assert st.temperatures[st.currentTemperature] == integrator.getTemperature()
    assert st.temperatures[st.currentTemperature] == simulation.context.getParameter('MonteCarloTemperature')*kelvin

#"Steps"	"Temperature (K)"	"200K Weight"	"216.012K Weight"	"233.306K Weight"	"251.984K Weight"	"272.158K Weight"	"293.947K Weight"	"317.48K Weight"	"342.898K Weight"	"370.35K Weight"	"400K Weight"
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
200.0 K
216.01194777846123 K
216.01194777846123 K
200.0 K
200.0 K
233.3058079152233 K
233.3058079152233 K
216.01194777846123 K
216.01194777846123 K
251.98420997897463 K
251.98420997897463 K
200.0 K
200.0 K
233.3058079152233 K
233.3058079152233 K
272.1580000348754 K
272.1580000348754 K
216.01194777846123 K
216.01194777846123 K
293.94689845511977 K
293.94689845511977 K
251.98420997897463 K
251.98420997897463 K
200.0 K
200.0 K
317.4802103936399 K
317.4802103936399 K
233.3058079152233 K
CPU times: user 21.2 s, sys: 2.1 s, total: 23.3 s
Wall time: 22.1 s
